# HER2 One Scanner - Aperio NIH

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_NIH/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'

batch_size = 32
num_classes = 3


## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(y)
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 5, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=5, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 2s 9ms/step - loss: 1.9409 - acc: 0.4855 - val_loss: 0.9756 - val_acc: 0.5714
Epoch 2/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9585 - acc: 0.5187 - val_loss: 0.9836 - val_acc: 0.5714
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9346 - acc: 0.5519 - val_loss: 0.9946 - val_acc: 0.6531
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0226 - acc: 0.4938 - val_loss: 0.9677 - val_acc: 0.5714
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9114 - acc: 0.6141 - val_loss: 1.0604 - val_acc: 0.2857
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9574 - acc: 0.5851 - val_loss: 1.3992 - val_acc: 0.2857
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8554 - acc: 0.6598 - val_loss: 0.7828 - val_acc: 0.7959
Epoch 8/1

241/241 [==============================] - 0s 1ms/step - loss: 0.3380 - acc: 0.8963 - val_loss: 0.2990 - val_acc: 0.8367
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2677 - acc: 0.8963 - val_loss: 0.2905 - val_acc: 0.8571
Epoch 63/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3718 - acc: 0.8506 - val_loss: 0.2576 - val_acc: 0.8571
Epoch 64/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2445 - acc: 0.8921 - val_loss: 0.2533 - val_acc: 0.8571
Epoch 65/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2351 - acc: 0.8963 - val_loss: 0.2010 - val_acc: 0.8571
Epoch 66/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.7588 - acc: 0.8091 - val_loss: 0.2628 - val_acc: 0.8571
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2443 - acc: 0.8963 - val_loss: 0.2029 - val_acc: 0.8571
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.0763 - acc: 0.9710 - val_loss: 0.0293 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1042 - acc: 0.9627 - val_loss: 0.0225 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0836 - acc: 0.9585 - val_loss: 0.0512 - val_acc: 0.9796
Epoch 124/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0485 - acc: 0.9917 - val_loss: 0.0192 - val_acc: 1.0000
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4490 - acc: 0.8880 - val_loss: 0.7288 - val_acc: 0.8367
Epoch 126/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2659 - acc: 0.9419 - val_loss: 0.0678 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0759 - acc: 0.9793 - val_loss: 0.0475 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.3979 - acc: 0.8838 - val_loss: 0.2689 - val_acc: 0.9167
Epoch 45/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4243 - acc: 0.8548 - val_loss: 0.2567 - val_acc: 0.9167
Epoch 46/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4406 - acc: 0.8465 - val_loss: 0.3429 - val_acc: 0.8750
Epoch 47/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4651 - acc: 0.8548 - val_loss: 0.2783 - val_acc: 0.9167
Epoch 48/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.6109 - acc: 0.7967 - val_loss: 0.3171 - val_acc: 0.9167
Epoch 49/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4560 - acc: 0.8548 - val_loss: 0.2694 - val_acc: 0.9167
Epoch 50/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5715 - acc: 0.7884 - val_loss: 0.3443 - val_acc: 0.9167
Epoch 51/1000
241/241 [==============================] - 0s 1ms/step 

Epoch 105/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.1639 - acc: 0.8340 - val_loss: 0.0725 - val_acc: 1.0000
Epoch 106/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1448 - acc: 0.9295 - val_loss: 0.0584 - val_acc: 1.0000
Epoch 107/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1189 - acc: 0.9544 - val_loss: 0.0413 - val_acc: 0.9792
Epoch 108/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1309 - acc: 0.9336 - val_loss: 0.0390 - val_acc: 1.0000
Epoch 109/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1090 - acc: 0.9336 - val_loss: 0.0562 - val_acc: 0.9792
Epoch 110/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0893 - acc: 0.9627 - val_loss: 0.0310 - val_acc: 1.0000
Epoch 111/1000
241/241 [==============================] - 0s 2ms/step - loss: 1.2643 - acc: 0.8257 - val_loss: 0.0861 - val_acc: 0.9792
Epoch 112/1000
241/241 [========================

...

241/241 [==============================] - 0s 1ms/step - loss: 0.1462 - acc: 0.9295 - val_loss: 0.0919 - val_acc: 0.9375
Epoch 132/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1348 - acc: 0.9129 - val_loss: 0.0531 - val_acc: 0.9583
Epoch 133/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1181 - acc: 0.9419 - val_loss: 0.0529 - val_acc: 1.0000
Epoch 134/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1693 - acc: 0.9004 - val_loss: 0.0756 - val_acc: 0.9167
Epoch 135/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1005 - acc: 0.9585 - val_loss: 0.1834 - val_acc: 0.9167
Epoch 136/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5015 - acc: 0.8797 - val_loss: 0.0707 - val_acc: 0.9583
Epoch 137/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1369 - acc: 0.9295 - val_loss: 0.0583 - val_acc: 0.9583
Epoch 138/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.0585 - acc: 0.9876 - val_loss: 0.0139 - val_acc: 1.0000
Epoch 192/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1427 - acc: 0.9461 - val_loss: 0.3451 - val_acc: 0.8542
Epoch 193/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1740 - acc: 0.9378 - val_loss: 0.0219 - val_acc: 1.0000
Epoch 194/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1035 - acc: 0.9710 - val_loss: 0.0149 - val_acc: 1.0000
Epoch 195/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0604 - acc: 0.9751 - val_loss: 0.0121 - val_acc: 1.0000
Epoch 196/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0685 - acc: 0.9710 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 197/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0618 - acc: 0.9793 - val_loss: 0.0081 - val_acc: 1.0000
Epoch 198/1000
241/241 [==============================] - 0s 2m

241/241 [==============================] - 0s 2ms/step - loss: 0.5952 - acc: 0.7925 - val_loss: 0.5441 - val_acc: 0.8125
Epoch 28/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5516 - acc: 0.8257 - val_loss: 0.4172 - val_acc: 0.8750
Epoch 29/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5247 - acc: 0.8257 - val_loss: 0.4195 - val_acc: 0.8542
Epoch 30/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5370 - acc: 0.8423 - val_loss: 0.5342 - val_acc: 0.8125
Epoch 31/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5136 - acc: 0.8257 - val_loss: 0.4950 - val_acc: 0.8125
Epoch 32/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5469 - acc: 0.8091 - val_loss: 0.6383 - val_acc: 0.7708
Epoch 33/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5078 - acc: 0.8299 - val_loss: 0.4579 - val_acc: 0.7708
Epoch 34/1000
241/241 [==============================] - 0s 1ms/step 

Epoch 88/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2168 - acc: 0.8838 - val_loss: 0.1031 - val_acc: 0.8958
Epoch 89/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1737 - acc: 0.9212 - val_loss: 0.2174 - val_acc: 0.8333
Epoch 90/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6872 - acc: 0.8216 - val_loss: 0.1199 - val_acc: 0.8958
Epoch 91/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1842 - acc: 0.9087 - val_loss: 0.1289 - val_acc: 0.8958
Epoch 92/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1662 - acc: 0.9295 - val_loss: 0.0985 - val_acc: 0.8958
Epoch 93/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1313 - acc: 0.9461 - val_loss: 0.1049 - val_acc: 0.9167
Epoch 94/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1413 - acc: 0.9212 - val_loss: 0.0800 - val_acc: 0.9583
Epoch 95/1000
241/241 [==============================] 

241/241 [==============================] - 0s 1ms/step - loss: 0.0566 - acc: 0.9710 - val_loss: 0.0097 - val_acc: 1.0000
Epoch 149/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0738 - acc: 0.9544 - val_loss: 0.6699 - val_acc: 0.8958
Epoch 150/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5148 - acc: 0.8921 - val_loss: 0.0399 - val_acc: 0.9792
Epoch 151/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0744 - acc: 0.9751 - val_loss: 0.0396 - val_acc: 0.9792
Epoch 152/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0949 - acc: 0.9585 - val_loss: 0.0158 - val_acc: 1.0000
Epoch 153/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0811 - acc: 0.9502 - val_loss: 0.0176 - val_acc: 1.0000
Epoch 154/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1522 - acc: 0.9212 - val_loss: 0.2821 - val_acc: 0.8958
Epoch 155/1000
241/241 [==============================] - 0s 1m